In [1]:
import scenvi 

In [2]:
scenvi 

<module 'scenvi' from '/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/__init__.py'>

In [3]:
import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)

In [4]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [10]:
seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
spatial_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

spatial_index = [list(spatial_data.obs['x_coord']), list(spatial_data.obs['y_coord'])]
spatial_data.obsm['spatial'] = np.array(spatial_index).T.astype('int')

seq_data

spatial_data

info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

import random 
random.seed(2023)
gene_for_impute = seq_data.var_names

seq_data = seq_data[:,gene_for_impute]
spatial_data = spatial_data[:,info_gene]

spatial_data_partial = spatial_data.copy()
seq_data = seq_data.copy()

seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars

# spatial_data_partial has a subset of the genes to train on
spatial_data_partial = spatial_data_partial

# spatial_data should use the same cells as our training data
# cells may have been removed by scanpy.pp.filter_cells()
spatial_data = spatial_data[spatial_data_partial.obs_names]


In [19]:
seq_data

AnnData object with n_obs × n_vars = 3005 × 19972
    obs: 'labels', 'precise_labels', 'cell_type', 'scClassify'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    layers: 'log'

In [21]:
seq_data

AnnData object with n_obs × n_vars = 3005 × 19972
    obs: 'labels', 'precise_labels', 'cell_type', 'scClassify'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    layers: 'log'

In [22]:
spatial_data_partial

AnnData object with n_obs × n_vars = 4530 × 20
    obs: 'x_coord', 'y_coord', 'labels', 'str_labels', 'batch', 'scClassify'
    obsm: 'spatial'

In [28]:
del seq_data.var['highly_variable']

In [29]:
from jax import random

In [30]:
import scenvi 

envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data, num_HVG = len(seq_data.var_names))

envi_model.train(key = random.key(0))
envi_model.impute_genes()
envi_model.infer_niche()

Computing Niche Covariance Matrices
Initializing VAE
Finished Initializing ENVI


/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)
 spatial: -1.189e+00 sc: -5.628e-01 cov: -2.597e-02 kl: 6.311e-01: 100%|██████████| 16000/16000 [07:31<00:00, 35.45it/s]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')


Finished imputing missing gene for spatial data! See 'imputation' in obsm of ENVI.spatial_data


In [31]:
envi_model.spatial_data.obsm['imputation']

,Aldoc,Anln,Apln,Cnr1,Crh,Crhbp,Ctps,Flt1,Gad2,Gfap,...,Mir875,B230206H07Rik,Igfbp4,Gm10696,Hrsp12,Necab3,Ffar2,Minpp1,Pqlc3,Crip2
0,3.890798,0.347196,0.052341,2.262364,0.267872,0.000168,1.583606,0.000122,2.354950,0.045816,...,0.000007,0.042727,0.542788,0.003667,0.990927,0.464624,0.000254,0.351401,0.023257,0.999909
1,3.034775,0.380003,1.178608,0.001539,0.000423,0.137021,0.249431,0.000288,0.414144,0.329378,...,0.000343,0.026172,0.564384,0.001401,0.825494,0.101939,0.000096,0.129188,0.005839,0.796457
2,3.027389,0.067541,0.031190,0.691702,1.113492,0.946629,0.698165,0.000443,25.102993,0.006727,...,0.000023,0.007454,1.168458,0.000780,0.588007,0.370951,0.000074,0.643475,0.005571,1.040304
3,0.016045,0.578078,0.037227,0.689628,0.000908,0.000456,0.311690,0.000440,1.887459,0.000234,...,0.000967,0.012055,1.329139,0.003252,0.569664,0.187886,0.000552,0.444005,0.009923,1.228038
4,0.000125,0.400275,0.086891,0.936333,0.069853,0.000190,0.640652,0.000448,3.183191,0.080496,...,0.000194,0.015705,0.741992,0.000805,0.500084,0.274491,0.000206,0.358943,0.004315,0.866431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4525,0.105373,0.001512,0.123752,0.002632,0.000280,0.000217,0.095735,0.000473,0.000663,0.000158,...,0.001707,0.006995,0.873256,0.003300,0.394379,0.320418,0.002141,0.294397,0.007594,1.328467
4526,0.066031,0.498194,0.183491,0.685272,0.001267,0.000592,0.393933,0.000930,0.261023,0.035086,...,0.000560,0.023158,1.027689,0.004708,0.569591,0.460157,0.001190,0.390571,0.014661,1.431250
4527,0.186408,0.580025,0.214355,0.224360,0.000451,0.000426,0.180878,0.000776,0.145249,0.154137,...,0.004270,0.006119,1.493771,0.001479,0.640848,0.170202,0.001114,0.412613,0.008854,1.563517
4528,0.111950,0.250422,0.046462,0.009308,0.123846,0.438341,0.276340,0.001434,0.192785,0.072704,...,0.003526,0.010444,1.096673,0.002762,0.429882,0.258029,0.001454,0.397249,0.011061,1.300701


In [32]:
seq_data.var_names

Index(['Tspan12', 'Tshz1', 'Fnbp1l', 'Adamts15', 'Cldn12', 'Rxfp1',
       '2310042E22Rik', 'Sema3c', 'Jam2', 'Apbb1ip',
       ...
       'Gm20826_loc1', 'Gm20826_loc2', 'Gm20877_loc2', 'Gm20877_loc1',
       'Gm20865_loc4', 'Gm20738_loc4', 'Gm20738_loc6', 'Gm21943_loc1',
       'Gm21943_loc3', 'Gm20738_loc3'],
      dtype='object', length=19972)

In [33]:
spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'].values


spatial_data_imputed = sc.AnnData(spatial_data.obsm['impute'], obs = spatial_data_partial.obs)
spatial_data_imputed.var_names = envi_model.spatial_data.obsm['imputation'].columns


#     spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/gimvigat_default_zsampleinference{i}.h5ad")
spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/alldatafolder/envi_imputesmfish_alldata.h5ad")

In [36]:
spatial_data_imputed

AnnData object with n_obs × n_vars = 4530 × 19972
    obs: 'x_coord', 'y_coord', 'labels', 'str_labels', 'batch', 'scClassify'

In [1]:
import scenvi 
import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)

2024-10-03 12:51:21.619925: W external/xla/xla/service/gpu/nvptx_compiler.cc:698] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
from scipy import stats
import scipy.stats as st
import scipy

In [3]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [16]:
seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
spatial_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")
seq_data.obs['scClassify'] = seq_data.obs['cell_type_alias_label2'].copy()
seq_data.var_names_make_unique()
spatial_data.obsm['spatial'] = spatial_data.obsm['X_spatial']

In [17]:
spatial_data

AnnData object with n_obs × n_vars = 160996 × 248
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'total_transciprts', 'total_genes', 'sizeFactor', 'scClassify', 'x_centroid', 'y_centroid'
    var: 'ID', 'Symbol', 'Type', 'total_cells', 'meanExprsPct_cells'
    uns: 'CellSPA', 'X_name'
    obsm: 'PCA', 'UMAP', 'X_spatial', 'spatial'
    layers: 'logcounts'

In [18]:
info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

/tmp/ipykernel_2059571/1732007094.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])


In [19]:
import random 
random.seed(2023)
gene_for_impute = seq_data.var_names

In [20]:
seq_data = seq_data[:,gene_for_impute]
spatial_data = spatial_data[:,info_gene]

In [21]:
spatial_data.layers['raw'] = spatial_data.X.toarray()

spatial_data.X = spatial_data.layers['raw'].copy()

spatial_data_partial = spatial_data.copy()
seq_data = seq_data.copy()

seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars

# spatial_data_partial has a subset of the genes to train on
spatial_data_partial = spatial_data_partial
spatial_data = spatial_data[spatial_data_partial.obs_names]

/tmp/ipykernel_2059571/2060916523.py:1: ImplicitModificationWarning: Setting element `.layers['raw']` of view, initializing view as actual.
  spatial_data.layers['raw'] = spatial_data.X.toarray()


In [22]:
seq_data.X = seq_data.X.toarray()
spatial_data.X = spatial_data.X.toarray()

In [23]:
spatial_data

View of AnnData object with n_obs × n_vars = 160996 × 165
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'total_transciprts', 'total_genes', 'sizeFactor', 'scClassify', 'x_centroid', 'y_centroid'
    var: 'ID', 'Symbol', 'Type', 'total_cells', 'meanExprsPct_cells'
    uns: 'CellSPA', 'X_name'
    obsm: 'PCA', 'UMAP', 'X_spatial', 'spatial'
    layers: 'logcounts', 'raw'

In [24]:
sc.pp.subsample(spatial_data, fraction=0.0001)

In [25]:
spatial_data

AnnData object with n_obs × n_vars = 16 × 165
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'total_transciprts', 'total_genes', 'sizeFactor', 'scClassify', 'x_centroid', 'y_centroid'
    var: 'ID', 'Symbol', 'Type', 'total_cells', 'meanExprsPct_cells'
    uns: 'CellSPA', 'X_name'
    obsm: 'PCA', 'UMAP', 'X_spatial', 'spatial'
    layers: 'logcounts', 'raw'

In [ ]:
from jax import random

import scenvi 

envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data, num_HVG = len(seq_data.var_names))

envi_model.train(epochs=1, key = random.key(0))
envi_model.impute_genes()
envi_model.infer_niche()

envi_model.spatial_data.obsm['imputation']

seq_data.var_names

spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'].values


spatial_data_imputed = sc.AnnData(spatial_data.obsm['impute'], obs = spatial_data_partial.obs)
spatial_data_imputed.var_names = envi_model.spatial_data.obsm['imputation'].columns


#     spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/gimvigat_default_zsampleinference{i}.h5ad")
spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_mousebrain/alldatafolder/envi_imputehumanbreast_alldata.h5ad")

spatial_data_imputed

In [1]:
import scenvi 
import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)

from scipy import stats
import scipy.stats as st
import scipy

2024-04-09 18:00:56.736664: W external/xla/xla/service/gpu/nvptx_compiler.cc:698] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

spatial_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/seqfishdata/data/ST/seqfish/seqfish_data.h5ad")
seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/seqfishdata/data/scRNAseq/seqfish/scRNAseq_seqfish.h5ad")

seq_data.obs['scClassify'] = seq_data.obs['celltype'].copy() 
spatial_data.obs['scClassify'] = spatial_data.obs['celltype_mapped_refined'].copy() 
seq_data.var_names_make_unique()

spatial_data.layers['raw'] = spatial_data.X.toarray()

spatial_data.X = spatial_data.layers['raw'].copy()

seq_data.obs['names'] = seq_data.obs_names
spatial_data.obs['names'] = spatial_data.obs_names

seq_data.obs['ind_x'] = seq_data.obs_names
spatial_data.obs['ind_x'] = spatial_data.obs_names


info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

import random 
random.seed(2023)
gene_for_impute = seq_data.var_names

seq_data = seq_data[:,gene_for_impute]
spatial_data = spatial_data[:,info_gene]

spatial_data_partial = spatial_data.copy()
seq_data = seq_data.copy()

seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars

# spatial_data_partial has a subset of the genes to train on
spatial_data_partial = spatial_data_partial

spatial_data = spatial_data[spatial_data_partial.obs_names]


In [3]:
seq_data.X = seq_data.X.toarray()
spatial_data.X = spatial_data.X.toarray()

In [4]:
spatial_data

View of AnnData object with n_obs × n_vars = 57536 × 177
    obs: 'uniqueID', 'embryo', 'pos', 'z', 'x_global', 'y_global', 'x_global_affine', 'y_global_affine', 'embryo_pos', 'embryo_pos_z', 'Area', 'UMAP1', 'UMAP2', 'celltype_mapped_refined', 'segmentation_vertices_x_global_affine', 'segmentation_vertices_y_global_affine', 'scClassify', 'names', 'ind_x'
    obsm: 'spatial'
    layers: 'normalized', 'raw'

In [5]:
# sc.pp.subsample(spatial_data, fraction=0.0001)

In [10]:
spatial_data.X

array([[0., 1., 0., ..., 4., 0., 2.],
       [2., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 2., 0., 0.],
       ...,
       [0., 2., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 2.],
       [0., 0., 1., ..., 0., 0., 2.]], dtype=float32)

In [9]:
from jax import random

import scenvi 

envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data, num_HVG = len(seq_data.var_names))

envi_model.train( key = random.key(0))
envi_model.impute_genes()
envi_model.infer_niche()

envi_model.spatial_data.obsm['imputation']

seq_data.var_names

spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'].values


spatial_data_imputed = sc.AnnData(spatial_data.obsm['impute'], obs = spatial_data_partial.obs)
spatial_data_imputed.var_names = envi_model.spatial_data.obsm['imputation'].columns


# #     spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/gimvigat_default_zsampleinference{i}.h5ad")
# spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_seqfish/alldatafolder/envi_seqfish_alldata.h5ad")

# spatial_data_imputed

Computing Niche Covariance Matrices


/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


Initializing VAE
Finished Initializing ENVI


 spatial: -8.312e-01 sc: -3.206e-01 cov: -7.935e-03 kl: 6.028e-01:   5%|▌         | 811/16000 [00:36<11:22, 22.25it/s] 


KeyboardInterrupt: 

In [8]:
spatial_data_imputed

AnnData object with n_obs × n_vars = 57536 × 29452
    obs: 'uniqueID', 'embryo', 'pos', 'z', 'x_global', 'y_global', 'x_global_affine', 'y_global_affine', 'embryo_pos', 'embryo_pos_z', 'Area', 'UMAP1', 'UMAP2', 'celltype_mapped_refined', 'segmentation_vertices_x_global_affine', 'segmentation_vertices_y_global_affine', 'scClassify', 'names', 'ind_x'

In [1]:
import scenvi 
import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)

2024-04-09 17:48:11.308643: W external/xla/xla/service/gpu/nvptx_compiler.cc:698] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
from scipy import stats
import scipy.stats as st
import scipy

In [3]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [4]:
spatial_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")
seq_data.obs['scClassify'] = seq_data.obs['graph_cluster_anno'].copy()
seq_data.var_names_make_unique()

/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [5]:
spatial_data.obsm['spatial'] = spatial_data.obsm['coord']

In [6]:
info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

/tmp/ipykernel_2067853/1732007094.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])


In [7]:
import random 
random.seed(2023)
gene_for_impute = seq_data.var_names

In [8]:
seq_data = seq_data[:,gene_for_impute]
spatial_data = spatial_data[:,info_gene]

In [9]:
spatial_data.layers['raw'] = spatial_data.X.toarray()

spatial_data.X = spatial_data.layers['raw'].copy()

spatial_data_partial = spatial_data.copy()
seq_data = seq_data.copy()

seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars

# spatial_data_partial has a subset of the genes to train on
spatial_data_partial = spatial_data_partial
spatial_data = spatial_data[spatial_data_partial.obs_names]

/tmp/ipykernel_2067853/2060916523.py:1: ImplicitModificationWarning: Setting element `.layers['raw']` of view, initializing view as actual.
  spatial_data.layers['raw'] = spatial_data.X.toarray()


In [10]:
seq_data.X = seq_data.X.toarray()
spatial_data.X = spatial_data.X.toarray()

In [11]:
spatial_data

View of AnnData object with n_obs × n_vars = 160254 × 281
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'total_transciprts', 'total_genes', 'sizeFactor', 'elongation', 'compactness', 'eccentricity', 'sphericity', 'solidity', 'convexity', 'circularity', 'density', 'scClassify', 'mean_cor_correlation', 'mean_celltype_correlation', 'mean_cor_cosine', 'mean_celltype_cosine', 'prop_detected_cor_correlation', 'prop_detected_celltype_correlation', 'prop_detected_cor_cosine', 'prop_detected_celltype_cosine', 'positive_F1', 'positive_Precision', 'positive_Recall', 'negative_F1', 'negative_Precision', 'negative_Recall', 'positive_exprsPct', 'negative_exprsPct', 'x_bin_groups', 'y_bin_groups', 'bin_groups'
    var: 'ID', 'Symbol', 'Type', 'total_cells', 'meanExprsPct_cells'
    uns: 'CellSPA', 'X_name'
    obsm: 'PCA', 'UMAP', 'coord', 'spatial'
    layers: 'logcounts', 'raw'

In [12]:
# sc.pp.subsample(spatial_data, fraction=0.0001)

In [13]:
spatial_data

View of AnnData object with n_obs × n_vars = 160254 × 281
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'total_transciprts', 'total_genes', 'sizeFactor', 'elongation', 'compactness', 'eccentricity', 'sphericity', 'solidity', 'convexity', 'circularity', 'density', 'scClassify', 'mean_cor_correlation', 'mean_celltype_correlation', 'mean_cor_cosine', 'mean_celltype_cosine', 'prop_detected_cor_correlation', 'prop_detected_celltype_correlation', 'prop_detected_cor_cosine', 'prop_detected_celltype_cosine', 'positive_F1', 'positive_Precision', 'positive_Recall', 'negative_F1', 'negative_Precision', 'negative_Recall', 'positive_exprsPct', 'negative_exprsPct', 'x_bin_groups', 'y_bin_groups', 'bin_groups'
    var: 'ID', 'Symbol', 'Type', 'total_cells', 'meanExprsPct_cells'
    uns: 'CellSPA', 'X_name'
    obsm: 'PCA', 'UMAP', 'coord', 'spatial'
    layers: 'logcounts', 'raw'

In [14]:
from jax import random

import scenvi 

envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data, num_HVG = len(seq_data.var_names))

envi_model.train(key = random.key(0))
envi_model.impute_genes()
envi_model.infer_niche()

envi_model.spatial_data.obsm['imputation']

seq_data.var_names

spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'].values


spatial_data_imputed = sc.AnnData(spatial_data.obsm['impute'], obs = spatial_data_partial.obs)
spatial_data_imputed.var_names = envi_model.spatial_data.obsm['imputation'].columns


#     spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/gimvigat_default_zsampleinference{i}.h5ad")
spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_humanbreast/alldatafolder/envi_imputehumanbreast_alldata.h5ad")

spatial_data_imputed

/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:383: UserWarning: `n_top_genes` > number of normalized dispersions, returning all genes with normalized dispersions.
  warnings.warn(msg, UserWarning)


Computing Niche Covariance Matrices


/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


Initializing VAE
Finished Initializing ENVI


 spatial: -1.680e+00 sc: -8.023e-01 cov: -1.902e+01 kl: 3.702e+00: 100%|██████████| 1/1 [00:27<00:00, 27.71s/it]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')
2024-04-09 17:50:56.300860: W external/tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 17.66MiB (rounded to 18515968)requested by op 
2024-04-09 17:50:56.301789: W external/tsl/tsl/framework/bfc_allocator.cc:497] ****************************************************************************************************
2024-04-09 17:50:56.301883: E external/xla/xla/pjrt/pjrt_stream_executor_client.cc:2732] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 18515968 bytes.
BufferAssignment

ValueError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 18515968 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:   35.32MiB
              constant allocation:         0B
        maybe_live_out allocation:   17.66MiB
     preallocated temp allocation:         0B
                 total allocation:   52.97MiB
              total fragmentation:         0B (0.00%)
Peak buffers:
	Buffer 1:
		Size: 17.66MiB
		Entry Parameter Subshape: f32[256,18082]
		==========================

	Buffer 2:
		Size: 17.66MiB
		Entry Parameter Subshape: f32[256,18082]
		==========================

	Buffer 3:
		Size: 17.66MiB
		Operator: op_name="jit(fn)/jit(main)/mul" source_file="/tmp/ipykernel_2067853/3202982017.py" source_line=8
		XLA Label: fusion
		Shape: f32[256,18082]
		==========================



In [ ]:
import scenvi 
import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)

from scipy import stats
import scipy.stats as st
import scipy

def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
spatial_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/spe_xenium.h5ad")
seq_data.obs['scClassify'] = seq_data.obs['cell_type_alias_label2'].copy()
seq_data.var_names_make_unique()
spatial_data.obsm['spatial'] = spatial_data.obsm['X_spatial']

spatial_data

info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

import random 
random.seed(2023)
gene_for_impute = seq_data.var_names

seq_data = seq_data[:,gene_for_impute]
spatial_data = spatial_data[:,info_gene]

spatial_data.layers['raw'] = spatial_data.X.toarray()

spatial_data.X = spatial_data.layers['raw'].copy()

spatial_data_partial = spatial_data.copy()
seq_data = seq_data.copy()

seq_gene_names = seq_data.var_names
n_genes = seq_data.n_vars

# spatial_data_partial has a subset of the genes to train on
spatial_data_partial = spatial_data_partial
spatial_data = spatial_data[spatial_data_partial.obs_names]

seq_data.X = seq_data.X.toarray()
spatial_data.X = spatial_data.X.toarray()

spatial_data

sc.pp.subsample(spatial_data, fraction=0.0001)

spatial_data





from jax import random

import scenvi 

envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data, num_HVG = len(seq_data.var_names))

envi_model.train(epochs=1, key = random.key(0))
envi_model.impute_genes()
envi_model.infer_niche()

envi_model.spatial_data.obsm['imputation']

seq_data.var_names

spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'].values


spatial_data_imputed = sc.AnnData(spatial_data.obsm['impute'], obs = spatial_data_partial.obs)
spatial_data_imputed.var_names = envi_model.spatial_data.obsm['imputation'].columns


#     spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/gimvigat_default_zsampleinference{i}.h5ad")
spatial_data_imputed.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_mousebrain/alldatafolder/envi_imputehumanbreast_alldata.h5ad")

spatial_data_imputed



In [11]:
import scenvi 

scenvi 

import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)
import jax

In [12]:
from scipy import stats
import scipy.stats as st
import scipy

In [13]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [16]:
# spatial_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
# seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")
import gc

In [18]:
for sample_index in range(0,10):
    seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")
    spatial_data = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/human_breast_simulation/spe_xenium_data_0.1_seed{sample_index}.h5ad")
    seq_data.var_names_make_unique()
    spatial_data.var_names_make_unique()
    spatial_data.X = spatial_data.X.toarray()
    seq_data.X = seq_data.X.toarray()
    seq_data.obs['names'] = [i for i in range(len(seq_data))]
    spatial_data.obs['names'] = [i for i in range(len(spatial_data))]
    
    seq_data.obs['ind_x'] = [i for i in range(len(seq_data))]
    spatial_data.obs['ind_x'] = [i for i in range(len(spatial_data))]
    spatial_data.obsm['spatial'] = spatial_data.obsm['coord']

    seq_data.obs['scClassify'] = seq_data.obs['graph_cluster_anno'].copy() 

    info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

    seq_data = seq_data[:,info_gene]
    spatial_data = spatial_data[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(spatial_data .var_names).intersection(seq_data.var_names))
    g1  = sorted(g1)
    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)
    spatial_data_partial = spatial_data[:, train_g].copy()
    seq_data = seq_data.copy()

    seq_gene_names = seq_data.var_names
    n_genes = seq_data.n_vars

    # spatial_data_partial has a subset of the genes to train on
    spatial_data_partial = spatial_data_partial

    spatial_data = spatial_data[spatial_data_partial.obs_names]
    
    envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data)
    envi_model.train(key = jax.random.key(sample_index))
    envi_model.impute_genes()
    envi_model.infer_niche()
    
    spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values

    gc.collect()
    spatial_data.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/envi_humanbreast_seed{sample_index}.h5ad")

/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/tmp/ipykernel_2322744/1732007094.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])


Computing Niche Covariance Matrices


/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


Initializing VAE
Finished Initializing ENVI


 spatial: -3.904e-01 sc: -3.651e-01 cov: -3.063e-03 kl: 7.041e-01: 100%|██████████| 16000/16000 [01:15<00:00, 213.03it/s]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')


...

/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


Initializing VAE
Finished Initializing ENVI


 spatial: -3.880e-01 sc: -3.581e-01 cov: -3.247e-03 kl: 7.047e-01: 100%|██████████| 16000/16000 [01:14<00:00, 213.73it/s]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')


Finished imputing missing gene for spatial data! See 'imputation' in obsm of ENVI.spatial_data


/tmp/ipykernel_2322744/3284622178.py:43: ImplicitModificationWarning: Setting element `.obsm['impute']` of view, initializing view as actual.
  spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values


In [19]:
seq_data

AnnData object with n_obs × n_vars = 30365 × 276
    obs: 'Barcode', 'sizeFactor', 'cluster', 'graph_cluster', 'ffpe_celltype', 'graph_cluster_anno', 'names', 'ind_x', 'scClassify'
    var: 'ID', 'Symbol', 'Type', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_pca', 'X_umap'
    layers: 'logcounts', 'log'

In [20]:
a  = 1

In [1]:
import scenvi 

scenvi 

import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)
import jax

from scipy import stats
import scipy.stats as st
import scipy

2024-04-09 01:03:26.566711: W external/xla/xla/service/gpu/nvptx_compiler.cc:698] The NVIDIA driver's CUDA version is 12.3 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [3]:
# spatial_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
# seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")

In [4]:
import gc

In [5]:
seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")

In [6]:
spatial_data = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_brain/spe_xenium_data_0.1_seed0.h5ad")

In [7]:
a = 2

In [ ]:
for sample_index in range(0,10):
    seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_brain/aibs_mouse_ctx-hpf_smartseq_sce.h5ad")
    spatial_data = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_brain/spe_xenium_data_0.1_seed{sample_index}.h5ad")
    seq_data.X = seq_data.X.toarray()
    seq_data.obs['scClassify'] = list(seq_data.obs['cell_type_alias_label2'])
    seq_data.obs['scClassify'] = seq_data.obs['scClassify'].astype('category')
    
    spatial_data.obsm['spatial'] = spatial_data.obsm['X_spatial']
    spatial_data.X = spatial_data.X.toarray()

    
    info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')
    seq_data = seq_data[:,info_gene]
    spatial_data = spatial_data[:,info_gene]
    
    seq_data.obs['names'] = [i for i in range(len(seq_data))]
    spatial_data.obs['names'] = [i for i in range(len(spatial_data))]

    import random 
    random.seed(2023)
    g1 = list(set(spatial_data.var_names).intersection(seq_data.var_names))
    g1  = sorted(g1)
    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

    spatial_data_partial = spatial_data[:, train_g].copy()
    seq_data = seq_data[:,g1]
    seq_data = seq_data.copy()

    seq_gene_names = seq_data.var_names
    n_genes = seq_data.n_vars

    spatial_data = spatial_data[spatial_data_partial.obs_names]


    envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data)
    envi_model.train(key = jax.random.key(sample_index))
    envi_model.impute_genes()
    envi_model.infer_niche()
    
    spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values

    gc.collect()
    spatial_data.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_brain/envi_mousebrain_seed{sample_index}.h5ad")


/tmp/ipykernel_757458/1732007094.py:23: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
/tmp/ipykernel_757458/3194952597.py:16: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  seq_data.obs['names'] = [i for i in range(len(seq_data))]
/tmp/ipykernel_757458/3194952597.py:17: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  spatial_data.obs['names'] = [i for i in range(len(spatial_data))]


Computing Niche Covariance Matrices


/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


Initializing VAE
Finished Initializing ENVI


 spatial: -5.695e-01 sc: -1.695e+00 cov: -5.089e-03 kl: 9.544e-01: 100%|██████████| 16000/16000 [01:45<00:00, 151.90it/s]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')


Finished imputing missing gene for spatial data! See 'imputation' in obsm of ENVI.spatial_data


In [7]:
import scenvi 

scenvi 

import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)
import jax

from scipy import stats
import scipy.stats as st
import scipy
import gc

In [8]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    overlap_gene = overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [9]:
# spatial_data =sc.read_h5ad("/gpfs/gibbs/pi/zhao/yl2687/data/spatial_fish_data/xenium_breast/spe_xenium_withMetrics.h5ad")
# seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/deconvdatasets/spatial_dataset/xenium_breast/sce_FFPE_full.h5ad")

In [10]:
for sample_index in range(0,10):
    seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/seqfishdata/data/scRNAseq/seqfish/scRNAseq_seqfish.h5ad")
    spatial_data = sc.read_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_seqfish/seqfish_data_seed{sample_index}.h5ad")
    seq_data.X = seq_data.X.toarray()
    spatial_data.X = spatial_data.X.toarray()
    seq_data.var_names_make_unique()
    
    seq_data.obs['scClassify'] = seq_data.obs['celltype'].copy() 
    spatial_data.obs['scClassify'] = spatial_data.obs['celltype_mapped_refined'].copy() 
    
    seq_data.obs['names'] = [i for i in range(len(seq_data))]
    spatial_data.obs['names'] = [i for i in range(len(spatial_data))]

    info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

    seq_data = seq_data[:,info_gene]
    spatial_data = spatial_data[:,info_gene]

    import random 
    random.seed(2023)
    g1 = list(set(spatial_data.var_names).intersection(seq_data.var_names))
    g1  = sorted(g1)
    train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

    spatial_data_partial = spatial_data[:, train_g].copy()
    seq_data = seq_data[:,g1]
    seq_data = seq_data.copy()

    seq_gene_names = seq_data.var_names
    n_genes = seq_data.n_vars

    spatial_data = spatial_data[spatial_data_partial.obs_names]


    envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data)
    envi_model.train(key = jax.random.key(sample_index))
    envi_model.impute_genes()
    envi_model.infer_niche()
    
    spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values

    gc.collect()
    spatial_data.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_seqfish/envi_seqfish_seed{sample_index}.h5ad")


Computing Niche Covariance Matrices


/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


Initializing VAE
Finished Initializing ENVI


 spatial: -4.195e-01 sc: -3.458e-01 cov: -6.932e-03 kl: 7.259e-01: 100%|██████████| 16000/16000 [01:40<00:00, 158.84it/s]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')


Finished imputing missing gene for spatial data! See 'imputation' in obsm of ENVI.spatial_data


...

/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


Initializing VAE
Finished Initializing ENVI


 spatial: -4.371e-01 sc: -3.405e-01 cov: -3.815e-03 kl: 7.203e-01: 100%|██████████| 16000/16000 [01:41<00:00, 157.17it/s]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')


Finished imputing missing gene for spatial data! See 'imputation' in obsm of ENVI.spatial_data


/tmp/ipykernel_2148352/1686206436.py:40: ImplicitModificationWarning: Setting element `.obsm['impute']` of view, initializing view as actual.
  spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values


In [22]:
import scenvi 

scenvi 

import scanpy as sc
from scipy import stats
import scipy.stats as st
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


import random 
random.seed(2023)
import jax

from scipy import stats
import scipy.stats as st
import scipy
import gc

In [23]:
def calcualte_pse_correlation(adata_sc, adata_st, celltype, p_value_threshold = 0.05, cor_threshold = 0.5):
    import random 
    random.seed(2023)
    overlap_gene = list(set(adata_sc.var_names).intersection(adata_st.var_names))
    overlap_gene = sorted(overlap_gene)
    adata_sc = adata_sc[:,overlap_gene]
    adata_st = adata_st[:,overlap_gene]
    
    cell_type_common = list(set(adata_sc.obs[celltype].unique()).intersection(adata_st.obs[celltype].unique()))
    
    pseudo_st = []
    pseudo_sc = []
    for i in cell_type_common:
        adata1 = adata_st[adata_st.obs[celltype] == i]
        adata2 = adata_sc[adata_sc.obs[celltype] == i]

        pseudo_st.append(np.mean(adata1.X.toarray(), axis = 0))
        pseudo_sc.append(np.mean(adata2.X.toarray(), axis = 0))
    
    pseudo_st = np.array(pseudo_st)
    pseudo_sc = np.array(pseudo_sc)

    cor_pearson = []
    cor_pvalue = []
    for i in range(pseudo_st.shape[1]):
        cor, pval = st.pearsonr(pseudo_st[:,i], pseudo_sc[:,i])
        cor_pearson.append(cor)
        cor_pvalue.append(pval)
        
    information_stat = pd.DataFrame()

    information_stat['pearson'] = cor_pearson
    information_stat['pvalue'] = cor_pvalue
    information_stat.index = adata_st.var_names

    information_stat_update = information_stat.loc[((information_stat['pvalue']<p_value_threshold) & (information_stat['pearson']>cor_threshold))]
    
    return information_stat_update.index

In [24]:
seq_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/scrnaseq_data.h5ad")
spatial_data = sc.read_h5ad("/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/spatial_data.h5ad")

In [25]:
info_gene = calcualte_pse_correlation(seq_data, spatial_data, 'scClassify')

In [26]:
seq_data = seq_data[:,info_gene]
spatial_data = spatial_data[:,info_gene]

In [27]:
# scanpy.pp.filter_cells(spatial_data, min_counts=1)
# scanpy.pp.filter_cells(seq_data, min_counts=1)

In [28]:
seq_data.obs['names'] = seq_data.obs_names
spatial_data.obs['names'] = spatial_data.obs_names

seq_data.obs['ind_x'] = seq_data.obs_names
spatial_data.obs['ind_x'] = spatial_data.obs_names

spatial_index = [list(spatial_data.obs['x_coord']), list(spatial_data.obs['y_coord'])]
spatial_data.obsm['spatial'] = np.array(spatial_index).T.astype('int')

/tmp/ipykernel_3525366/2654861310.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  seq_data.obs['names'] = seq_data.obs_names
/tmp/ipykernel_3525366/2654861310.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  spatial_data.obs['names'] = spatial_data.obs_names


In [29]:
import random 
random.seed(2023)
g1 = list(set(spatial_data.var_names).intersection(seq_data.var_names))
g1 = sorted(g1)
train_g, test_g = train_test_split(g1, test_size=0.33, random_state=2023)

In [30]:
# spatial_data_old = spatial_data.copy()
scrna_data_old = seq_data.copy()

In [32]:
# for prop in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
#     sample_index = 0
#     spatial_data = spatial_data_old.copy()

#     sc.pp.subsample(spatial_data, fraction = prop, random_state = 0)
#     print(spatial_data)
#     spatial_data_partial = spatial_data[:, train_g].copy()
#     seq_data = seq_data.copy()

#     seq_gene_names = seq_data.var_names
#     n_genes = seq_data.n_vars
    
#     # spatial_data_partial has a subset of the genes to train on
#     spatial_data_partial = spatial_data_partial

#     spatial_data = spatial_data[spatial_data_partial.obs_names]
    
#     envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data)
#     envi_model.train(key = jax.random.key(sample_index))
#     envi_model.impute_genes()
#     envi_model.infer_niche()
    
#     spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values

#     gc.collect()
#     spatial_data.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/envi_changespatial_{prop}.h5ad")

for prop in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    sample_index = 0
    seq_data = scrna_data_old.copy()

    sc.pp.subsample( seq_data, fraction = prop, random_state = 0)
    print(seq_data)
    spatial_data_partial = spatial_data[:, train_g].copy()
    seq_data = seq_data.copy()

    seq_gene_names = seq_data.var_names
    n_genes = seq_data.n_vars
    
    # spatial_data_partial has a subset of the genes to train on
    spatial_data_partial = spatial_data_partial

    spatial_data = spatial_data[spatial_data_partial.obs_names]
    
    envi_model = scenvi.ENVI(spatial_data = spatial_data_partial, sc_data = seq_data)
    envi_model.train(key = jax.random.key(sample_index))
    envi_model.impute_genes()
    envi_model.infer_niche()
    
    spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values

    gc.collect()
    spatial_data.write_h5ad(f"/gpfs/gibbs/pi/zhao/tl688/tangram/data_smfish/envi_changernaseq_{prop}.h5ad")
    

AnnData object with n_obs × n_vars = 300 × 20
    obs: 'labels', 'precise_labels', 'cell_type', 'scClassify', 'names', 'ind_x'
Computing Niche Covariance Matrices
Initializing VAE
Finished Initializing ENVI


/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)
 spatial: -1.287e+00 sc: -4.296e-01 cov: -3.546e-02 kl: 6.690e-01: 100%|██████████| 16000/16000 [01:06<00:00, 240.10it/s]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')


Finished imputing missing gene for spatial data! See 'imputation' in obsm of ENVI.spatial_data


/tmp/ipykernel_3525366/4106947776.py:50: ImplicitModificationWarning: Setting element `.obsm['impute']` of view, initializing view as actual.
  spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


AnnData object with n_obs × n_vars = 601 × 20
    obs: 'labels', 'precise_labels', 'cell_type', 'scClassify', 'names', 'ind_x'
Computing Niche Covariance Matrices
Initializing VAE
Finished Initializing ENVI


...

/tmp/ipykernel_3525366/4106947776.py:50: ImplicitModificationWarning: Setting element `.obsm['impute']` of view, initializing view as actual.
  spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/utils.py:226: ImplicitModificationWarning: Setting element `.layers['log']` of view, initializing view as actual.
  spatial_data.layers['log'] = np.log(spatial_data.X+1)


AnnData object with n_obs × n_vars = 2704 × 20
    obs: 'labels', 'precise_labels', 'cell_type', 'scClassify', 'names', 'ind_x'
Computing Niche Covariance Matrices
Initializing VAE
Finished Initializing ENVI


 spatial: -1.287e+00 sc: -5.517e-01 cov: -3.540e-02 kl: 6.556e-01: 100%|██████████| 16000/16000 [01:07<00:00, 237.30it/s]
/gpfs/gibbs/project/zhao/tl688/conda_envs/envi/lib/python3.9/site-packages/scenvi/ENVI.py:416: ImplicitModificationWarning: Setting element `.obsm['envi_latent']` of view, initializing view as actual.
  self.sc_data.obsm['envi_latent'] = self.encode(self.sc_data[:, self.spatial_data.var_names].X, mode = 'sc')


Finished imputing missing gene for spatial data! See 'imputation' in obsm of ENVI.spatial_data


/tmp/ipykernel_3525366/4106947776.py:50: ImplicitModificationWarning: Setting element `.obsm['impute']` of view, initializing view as actual.
  spatial_data.obsm['impute'] = envi_model.spatial_data.obsm['imputation'][list(spatial_data.var_names)].values


In [33]:
spatial_data_partial

AnnData object with n_obs × n_vars = 4530 × 13
    obs: 'x_coord', 'y_coord', 'labels', 'str_labels', 'batch', 'scClassify', 'names', 'ind_x'
    obsm: 'spatial', 'impute'